In [1]:
import pandas as pd
import numpy as np
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
stop_words = stopwords.words('russian')
m = MorphAnalyzer()

### Подготовка корпуса

Все тексты взяты с habr.com, в качестве ключевых слов использованы теги, проставленные авторами текстов. Всего в корпусе 6 текстов.

Получилось так, что эталонных ключевых слов как правило больше, чем слов из источника. Зачастую эталонные ключевые слова и слова из источника пересекаются, но я реже включала имена собственные (например, названия фирм или университетов) в свой список. Я не включала в свой список и те слова, которые не встречаются в тексте, но которые автор выделил как тег.

In [3]:
def get_text(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
    texts = text.split('\n\n')
    return texts

In [4]:
def get_keywords(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        text = f.read()
    lines = text.split('\n')
    keywords = []
    my_keywords = []
    for line in lines:
        keywords.append(line.split(';')[0])
        my_keywords.append(line.split(';')[1])
    return keywords, my_keywords

In [5]:
def lemmatize_keywords(keyword_line):
    lemmatized = []
    keywords = keyword_line.split(',')
    for word in keywords:
        parts = word.split(' ')
        norm_parts = [m.parse(p)[0].normal_form for p in parts]
        lemmatized.append(' '.join(norm_parts))
    return ','.join(lemmatized)

In [6]:
texts = get_text('habr_texts.txt')
source_keywords, my_keywords = get_keywords('habr_tags.txt')
my_keywords = [lemmatize_keywords(words) for words in my_keywords]
df = pd.DataFrame(texts, columns = ['text'])
df['keywords'] = source_keywords
df['my_keywords'] = my_keywords
df['source'] = ['habr.com']*len(texts)
df.head()

,text,keywords,my_keywords,source
0,"По сведениям, которые интернет-издательство Bl...","распознавание голоса,голосовые ассистенты,расп...","amazon,патент,эмоция,голос",habr.com
1,Недавно на шоссе меня подрезал таксист. Я без ...,"языки,родной язык,иностранный язык,эмоции,логи...","язык,эмоция,ругательство,билингвизм,родный язы...",habr.com
2,Создатели робота-музыканта Shimon анонсировали...,"роботы,робототехника,университет джорджии,geor...","робот,музыка,песня,shimon,альбом",habr.com
3,YouTube удалил из открытого доступа ряд видеоз...,"роботы,битвы роботов,цензура,эмпатия,сочувствие","робот,битва,цензура,эмпатия,сочувствие,youtube",habr.com
4,Идея сделать робота максимально похожим на чел...,"Toshiba,робот,андроид,искусственный интеллект","робот,человекоподобный робот,антропоморфный ро...",habr.com


Тексты и эталонные ключевые слова лемматизированы.

In [7]:
def lemmatize(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(m.parse(t)[0].normal_form)
    return ' '.join(lemmas)

In [8]:
lemma_texts = [lemmatize(text) for text in df['text'].values]

### RAKE

In [9]:
rake = RAKE.Rake(stop_words)

In [10]:
rake_words = [rake.run(lemma_text, maxWords=2, minFrequency=2) for lemma_text in lemma_texts]

for i, words in enumerate(rake_words):
    rake_words[i] = [word[0] for word in rake_words[i]]

In [11]:
print('Rake:', ','.join(rake_words[4]))
print('Эталон:', df['my_keywords'].iloc[4])

Rake: человекоподобный робот,уметь ходить,робот,уметь,айко,похожий,человек,u,r,рассказывать,работать,телевокс,представить,глаз,—
Эталон: робот,человекоподобный робот,антропоморфный робот,история,технология,внешность,функция


### TextRank

In [17]:
textrank_words = [keywords.keywords(lemma_text, language='russian', 
                  additional_stopwords=stop_words, scores=True, ratio=0.05) for lemma_text in lemma_texts]

for i, words in enumerate(textrank_words):
    textrank_words[i] = [word[0] for word in textrank_words[i]]

In [18]:
print('TextRank:', ','.join(textrank_words[4]))
print('Эталон:', df['my_keywords'].iloc[4])

TextRank: робот,человек,годиться,год,это,очень,уровень,телевокс мочь,способный,atlas,телевокса,движение,стационарный,статья,стать,айко,любой,honda,голос,человеческий,самый,современный,современность,мимика,антропоморфность,антропоморфный,универсальный
Эталон: робот,человекоподобный робот,антропоморфный робот,история,технология,внешность,функция


### TF-IDF

Включаю биграммы, беру только top-10

In [19]:
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))
tfidf = vectorizer.fit_transform(lemma_texts)

In [20]:
def get_tfidf_keyword(vectors):
    features = np.array(vectorizer.get_feature_names())
    sorted_indices = np.argsort(vectors.toarray()).ravel()[::-1]
    return features[sorted_indices][:10]

In [21]:
tfidf_words = []
for i in range(len(lemma_texts)):
    keywords = get_tfidf_keyword(tfidf[i])
    tfidf_words.append(keywords)

In [22]:
print('Tf-Idf:', ','.join(tfidf_words[4]))
print('Эталон:', df['my_keywords'].iloc[4])

Tf-Idf: робот,человек,человекоподобный робот,человекоподобный,год,очень,уметь,похожий человек,айко,универсальный
Эталон: робот,человекоподобный робот,антропоморфный робот,история,технология,внешность,функция


### Извлечение частеречных шаблонов

Среди эталонных слов не такое большое разнообразие сочетаний. Встречаются существительные, слова, записанные латиницей и сочетания прилагательных с существительными.

In [23]:
def get_pos_pattern(word_line):
    tokens = word_line.split(' ')
    pos = [m.parse(t)[0].tag.POS if 'LATN' not in m.parse(t)[0].tag else 'LATN' for t in tokens]
    if None in pos:
        return ''
    return '+'.join(pos)

In [24]:
def get_gold_keyword_patterns(df):
    pos_combos = []
    for keywords in df['my_keywords'].values:
        lines = keywords.split(',')
        for line in lines:
            pos_combos.append(get_pos_pattern(line))
    return set(pos_combos)

In [25]:
pos_patterns = get_gold_keyword_patterns(df)
pos_patterns

{'ADJF+NOUN', 'LATN', 'NOUN'}

In [26]:
def filter_with_pattern(keywords, pos_patterns):
    filtered = []
    for line in keywords:
        pos = get_pos_pattern(line)
        if pos in pos_patterns:
            filtered.append(line)
    return filtered

In [27]:
rake_filtered = [filter_with_pattern(words, pos_patterns) for words in rake_words]
textrank_filtered = [filter_with_pattern(words, pos_patterns) for words in textrank_words]
tfidf_filtered = [filter_with_pattern(words, pos_patterns) for words in tfidf_words]

In [28]:
print('Rake filtered:', ','.join(rake_filtered[4]), '\n')
print('TextRank filtered:', ','.join(textrank_filtered[4]), '\n')
print('Tfidf filtered:', ','.join(tfidf_filtered[4]), '\n')
print('Эталон:', df['my_keywords'].iloc[4])

Rake filtered: человекоподобный робот,робот,айко,человек,u,r,телевокс,глаз 

TextRank filtered: робот,человек,год,уровень,atlas,телевокса,движение,статья,айко,honda,голос,современность,мимика,антропоморфность 

Tfidf filtered: робот,человек,человекоподобный робот,год,похожий человек,айко 

Эталон: робот,человекоподобный робот,антропоморфный робот,история,технология,внешность,функция


### Оценка без фильтров

Посчитаем точность, полноту, F-меру для полученных тремя методами слов, к которым еще не был применен фильтр

In [29]:
def evaluate_one_set(selected, gold):
    a = set(selected)
    b = set(gold)
    precision = len(a & b) / len(a)
    recall = len(a & b) / len(b)
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    return precision, recall, f1

In [30]:
def evaluate(method_words, gold):
    all_prec, all_rec, all_f1 = 0, 0, 0
    for i, words in enumerate(method_words):
        precision, recall, f1 = evaluate_one_set(words, gold[i])
        all_prec += precision
        all_rec += recall
        all_f1 += f1
    return all_prec/len(gold), all_rec/len(gold), all_f1/len(gold)

In [31]:
gold = [words.split(',') for words in df['my_keywords'].values]

In [39]:
prec, rec, f1 = evaluate(rake_words, gold)
print(f'RAKE: \t\t precision {prec:0.2} \t recall {rec:0.2} \t F1 {f1:0.2}')

prec, rec, f1 = evaluate(textrank_words, gold)
print(f'TextRank: \t precision {prec:0.2} \t recall {rec:0.2} \t F1 {f1:0.2}')

prec, rec, f1 = evaluate(tfidf_words, gold)
print(f'Tf-Idf: \t precision {prec:0.2} \t recall {rec:0.2} \t F1 {f1:0.2}')

RAKE: 		 precision 0.11 	 recall 0.25 	 F1 0.15
TextRank: 	 precision 0.12 	 recall 0.35 	 F1 0.18
Tf-Idf: 	 precision 0.22 	 recall 0.42 	 F1 0.28


* Лучшие результаты по всем параметрам показал Tf-idf - он вычленял достаточно нужных слов и мало ненужных. Возможно, повлияло и то, что я сама установила сколько слов мне нужно - всего 10, потому что знаю, что эталонных слов больше не бывает. 
* Изначально TextRank показал худшие результаты, потому что предлагал избыточное количество слов. У него была очень высокая полнота, но очень низкая точность. Кроме того, TextRank, по-видимому, не выделяет биграммы, что уменьшает точность. Но после того, как я уменьшила параметр ratio, количество предлагаемых слов уменьшилось, следовательно, точность увеличилась.
* В итоге худший результат оказался у Rake. 

### Оценка с фильтром

У всех методов качество улучшилось, что было ожидаемо. 

Теперь F1 у TextRank и Rake одинаковы, но TextRank достигает этой цифры за счет предложения большего количества слов, а Rake за счет точности, в чем ему помогает умение выделять биграммы.

In [40]:
prec, rec, f1 = evaluate(textrank_filtered, gold)
print(f'TextRank with filter: \t precision {prec:0.2} \t recall {rec:0.2} \t F1 {f1:0.2}')

prec, rec, f1 = evaluate(rake_filtered, gold)
print(f'RAKE with filter: \t precision {prec:0.2} \t recall {rec:0.2} \t F1 {f1:0.2}')

prec, rec, f1 = evaluate(tfidf_filtered, gold)
print(f'Tf-Idf with filter: \t precision {prec:0.2} \t recall {rec:0.2} \t F1 {f1:0.2}')

TextRank with filter: 	 precision 0.19 	 recall 0.35 	 F1 0.24
RAKE with filter: 	 precision 0.23 	 recall 0.25 	 F1 0.24
Tf-Idf with filter: 	 precision 0.32 	 recall 0.42 	 F1 0.35


### Анализ

Оценка набора ключевых слов довольно субъективна. Уверена, что нашлись бы люди, которым больше бы понравился набор, предложенный тремя автоматическими способами, чем мной. Я тоже готова согласиться с некоторым их словами. 

Основная проблема данных трех методов в том, что они предлагают очень много слов. Все же не все слова, которые часто встречаются являются ключевыми. Иногда некоторые слова труднее заменить на синонимы, поэтому они так часто употребляются. Возможно, с этой проблемой помогло бы считать косинусную близость между выделенными ключевыми словами. Слова, которые совсем далеки от остальных, скорее всего попали в набор по ошибке.  
Например, в примере ниже сочетание "минное поле" выделилось как ключевое слово. Оно часто встречалось, но лишь потому что его никак нельзя заменить в рассказе. А вот к понятию робот может относится много других понятий. 


Мне, как человеку, свойственно выделять ключевые слова не с помощью подсчета. Иногда эталонные ключевые слова очень редко встречались в тексте, а появлялись лишь потому что я сопосбна к обобщению. Эти же методы не могут выдать слово, которое в тексте не встречалось, потому что не могут обобщить несколько понятий под одним термином. С этим мог бы помочь WordNet, из которого можно извлекать гиперонимы для слов.  
Например, если бы у нас был текст про породы собак и их названия часто там употреблялись, а само слово "собака" не встречалось бы, то с помощью гиперонима некоторая система все равно смогла бы предложить это слово в качестве ключевого.


In [41]:
print('Rake filtered:', ','.join(rake_filtered[3]), '\n')
print('TextRank filtered:', ','.join(textrank_filtered[3]), '\n')
print('Tfidf filtered:', ','.join(tfidf_filtered[3]), '\n')
print('Эталон:', df['my_keywords'].iloc[3])

Rake filtered: минный поле,youtube,человек,робот,животное,общество 

TextRank filtered: робот,человек,google,youtube,контент,поле,пол,животное,компания,год,цензура,система,дарлинг,исследование,фильтрация 

Tfidf filtered: робот,youtube,google,фильтрация,зрение,видеоролик,контент,человек 

Эталон: робот,битва,цензура,эмпатия,сочувствие,youtube
